In [30]:
import csv 
import requests
import os
from openai import OpenAI
from bs4 import BeautifulSoup
from typing import Dict, List
from urllib.parse import urlparse

In [1]:
client = OpenAI(
    api_key = os.environ.get('Secret API key'),
)

NameError: name 'OpenAI' is not defined

In [25]:
def scrape_websites_from_csv(csv_file_path: str) -> List[Dict[str, str]]:
# Function to scrape CSV and domains in it -> to return parsed HTML content.
    scraped_data = []

    with open(csv_file_path, 'r') as csvfile:
        csv_reader = csv.reader(csvfile)
        next(csv_reader, None)  # Skip the header row if it exists

        for row in csv_reader:
            if not row:  # Skip empty rows
                continue
            
            url = row[0].strip()  # Assuming the URL is in the first column
            if not url.startswith('http'):
                url = 'http://' + url

            try:
                response = requests.get(url, timeout=10)
                soup = BeautifulSoup(response.content, 'html.parser')
                
                data = {
                    'url': url,
                    'domain': urlparse(url).netloc,
                    'title': soup.title.string if soup.title else 'No title found',
                    'meta_description': soup.find('meta', attrs={'name': 'description'})['content'] if soup.find('meta', attrs={'name': 'description'}) else 'No meta description found',
                    'headings': [h.text for h in soup.find_all(['h1', 'h2', 'h3'])[:5]],
                    'paragraphs': [p.text for p in soup.find_all('p')[:3]]
                }
                
                scraped_data.append(data)
                print(f"Successfully scraped: {url}")
            
            except Exception as e:
                print(f"Error scraping {url}: {str(e)}")
                scraped_data.append({
                    'url': url,
                    'domain': urlparse(url).netloc,
                    'error': str(e)
                })

    return scraped_data

In [26]:
def analyze_content(data: Dict[str, str]) -> List[str]:
    #Analyze scraped content to identify key themes and topics.
    relevant_content = []
    
    keywords = ['achievement','professional','community','remote work', 'solution', 'connection', 'recognition', 'retention'] #identify content that matches Litespaces's values
    
    all_text = ' '.join(data['headings'] + data['paragraphs']).lower()
    
    for keyword in keywords:
        if keyword in all_text:
            relevant_content.append(f"Website mentions {keyword}")
    
    return relevant_content

In [33]:
def generate_personalized_email(data: Dict[str, str], relevant_content: List[str]) -> str:
    #Generate a personalized email for prospects

    company_name = data['domain'].split('.')[0].capitalize() #Extract company name from domain
    content_string = '. '.join(relevant_content)

    prompt = f"""
    Write a personalized outbound sales email for a startup called Litespace to {company_name}.
    Use the following information about their website content:
    {content_string}

    Litespace is a startup that offers innovative workspace solutions to improve productivity and collaboration in remote work environments.
    Litespace's core values are: inspire connection, build community, peer recognition, increase retention.

    The email should:
    1. Be professional and engaging
    2. Highlight how Litespace's values align with the content found on {company_name}'s website
    3. Suggest how Litespace could benefit their work based on the identified relevant content
    4. Include a call to action to schedule a brief call

    Keep the email concise, under 200 words.
    """
    try:
        response = openai.Completion.create(
            model="gpt-3.5",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that writes personalized outbound sales emails for Litespace."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=200,
            n=1,
            stop=None,
            temperature=0.7 
            )

        generated_email = response.choices[0].message.content.strip()
        return generated_email

    except Exception as e:
        print(f"Error generating email: {str(e)}")
        return f"Error generating email for {company_name}: {str(e)}"

In [28]:
def main():
    csv_file_path = input("Enter the path to your CSV file containing website URLs: ")
    scraped_websites = scrape_websites_from_csv(csv_file_path)
    #Scrape domains in CSV file

    # Process each scraped website
    for website_data in scraped_websites:
        if 'error' in website_data:
            print(f"Skipping {website_data['url']} due to error: {website_data['error']}")
            continue

        relevant_content = analyze_content(website_data)
        personalized_email = generate_personalized_email(website_data, relevant_content)

        print(f"\nGenerated Personalized Email for {website_data['domain']}:")
        print("-----------------------------")
        print(personalized_email)
        print("\n")

if __name__ == "__main__":
    main()

Successfully scraped: https://www.litespace.io/
Successfully scraped: https://www.humi.ca/
Successfully scraped: https://www.deel.com/
Successfully scraped: https://www.oyster.com/
Error generating email: 

You tried to access openai.Completion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


Generated Personalized Email for www.litespace.io:
-----------------------------
Error generating email for Www: 

You tried to access openai.Completion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade 